In [1]:
from whispr import *

In [2]:
# gene expression. 2.5ul rxns, 75% txtl
rxn_vol = 3
source_plate_type = '384PP_AQ_BP' 

sp_plasmids_file = 'source_plates/221018_arpae_plasmids.xlsx'
sp_plasmids = pd.read_excel(sp_plasmids_file, index_col = 0)
sp_plasmids = sp_plasmids[~sp_plasmids['Well'].isna()]

filename = '221018_arpae_genex.csv'
layout_genex_file = 'plate_layouts/'+filename 
layout_genex = pd.read_csv(layout_genex_file, index_col = 0, dtype = str)

mt_genex_file = 'mixing_tables/221018_arpae_plasmids.csv'
mt_genex = pd.read_csv(mt_genex_file, index_col = 0, dtype = str).fillna(0)

checkInputs(sp_plasmids, mt_genex,source_plate_type)

vol_table_df = generateVolumeTable(mt_genex, sp_plasmids, rxn_vol = 0.25*rxn_vol, total_vol = rxn_vol, fill_with='water', multiRpW = True)

#specify rxn_vol (default = 2.5) and total_vol (default = 10) if you'd like to change the volume of each individual replicate or the total reaction volume
protocol_genex_df = writeProtocol(source_plate_type, vol_table_df, layout_genex,sp_plasmids, update_source_vol='source_plates/221018_arpae_plasmids_updated.xlsx')


Volumes used from each well for this protocol:
{'1_A1': 22.3, '1_P1': 2.85, '1_P2': 3.8, '1_P3': 5.7, '1_P5': 2.85}


In [3]:
# everything gets txtl

# get destination wells
for layout_id, layout in enumerate([layout_genex]):    
    # reads plate layout and assigns wells to each reaction (rxn_loc; dict)
    labels = pd.unique(np.concatenate(layout.values))
    rxn_loc = {}
    for l in labels:
        if type(l) is str:
            index = layout[layout.isin([l])].stack().index
            rxn_loc[l] = []
            for i in index:
                rxn_loc[l].append(str(i[0]) + str(i[1]))


txtl_well = 'P24'
for label, dwell in rxn_loc.items():
    txtl_vol = vol_table_df[vol_table_df['Label'] == label].iloc[0,1:].sum()/0.25*0.75*1000 # add txtl to make 75%
    row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': txtl_well,
                                'Destination Plate Name':'Destination[1]', 'Destination Well': dwell[0], 'Transfer Volume': txtl_vol}
    protocol_genex_df = protocol_genex_df.append(row, ignore_index = True)

protocol_genex_df.to_csv('protocols/'+filename,index = False)

# carry out gene expression reaction

TypeError: append() takes no keyword arguments

In [66]:
protocol_genex_df[protocol_genex_df['Source Well'] == 'P24']['Transfer Volume'].sum()/1000

112.49999999999999

In [ ]:
# dilution. everything gets Tris, for 20x -> 57uL
protocol_dilution_df = pd.DataFrame(columns=protocol_genex_df.columns)
source_plate_type = '6RES_AQ_B' 
tris_well = 'A1'
tris_vol = 57*1000

for dwell in protocol_genex_df['Destination Well'].unique():
    row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': tris_well,
                                'Destination Plate Name':'Destination[1]', 'Destination Well': dwell, 'Transfer Volume': tris_vol}
    protocol_dilution_df = protocol_dilution_df.append(row, ignore_index = True)

protocol_dilution_df.to_csv('protocols/221004_arpae_dilutions.csv',index = False)

In [ ]:
# from source plate with buffer, mix into 96

# from source plate with genex, mix the diluted txtl

In [3]:
# biosynthesis. 25ul rxns, 2.5ul of diluted txtl

# buffer source plate
sp_buffers_file = 'source_plates/221018_arpae_buffers.xlsx'
sp_buffers= pd.read_excel(sp_buffers_file, index_col = 0)
sp_buffers = sp_buffers[~sp_buffers['Well'].isna()]

# HEPES reservoir (actually same plate as Tris)
sp_hepes = pd.DataFrame(columns=sp_buffers.columns)
sp_hepes = sp_hepes.append({'Label':'HEPES', 'Well':'A2,A3', 'Concentration':'', 'Volume':'2000,2000'}, ignore_index = True)

# gene expression source plate (diluted w/ Tris)
sp_genex = sp_from_layout(layout_genex, 60)

# group source plates
sp_types = ['384PP_AQ_BP','6RES_AQ_BP2','384PP_AQ_BP'] # triple check this
sps = [sp_buffers,sp_hepes,sp_genex]

# get mixing table
mt_biosyn_file = 'mixing_tables/221018_arpae_buffers.csv'
mt_biosyn = pd.read_csv(mt_biosyn_file, index_col = 0, dtype = str).fillna(0)

# check formats
checkInputs(sps,mt_biosyn,sp_types)

# layouts for destination plate(s)
filename = '221018_arpae_biosyn.csv'
layout_biosyn1_file = 'plate_layouts/'+filename 
layout_biosyn1 = pd.read_csv(layout_biosyn1_file, index_col = 0, dtype = str)
# filename = '221018_arpae_biosyn_neg.csv'
# layout_biosyn2_file = 'plate_layouts/'+filename 
# layout_biosyn2 = pd.read_csv(layout_biosyn2_file, index_col = 0, dtype = str)
# layouts = [layout_biosyn1,layout_biosyn2]

vol_table_df = generateVolumeTable(mt_biosyn, sps, rxn_vol = 22.5, total_vol = 25, fill_with='HEPES')

protocol_biosyn_dfs = writeProtocol(sp_types, vol_table_df, layout_biosyn1,sps, update_source_vol='source_plates/221018_arpae_combined_sp_updated.xlsx')

#protocol_biosyn_df.to_csv('protocols/221018_arpae_biosyn.csv',index = False)

Volumes used from each well for this protocol:
{'1_A1': 28.8, '1_B1': 12.0, '1_C1': 12.0, '1_D1': 24.0, '1_E1': 40.0, '1_E2': 40.0, '1_E3': 40.0, '1_F1': 4.8, '1_G1': 19.2, '1_H1': 41.0, '1_H2': 7.0, '1_I1': 36.0, '1_J1': 24.0, '1_K1': 41.9, '1_K2': 8.0, '1_L1': 1.9, '1_M1': 6.0, '2_A2': 773.4, '3_N2': 30.0, '3_N3': 30.0, '3_O2': 30.0, '3_O3': 30.0, '3_P2': 12.5, '3_P3': 12.5}


In [6]:
protocol_biosyn_dfs[0].to_csv('protocols/221018_arpae_biosyn_384.csv',index = False)
protocol_biosyn_dfs[1].to_csv('protocols/221018_arpae_biosyn_6RES.csv',index = False)